# Development Scratch

This notebook contains ideas/scratch/notes for development

In [1]:
import os
import sys
from distutils.util import strtobool

PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, PWD)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [2]:
import datetime
import django
import pandas as pd

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.models import *
from events.utils import get_all_event_participants
from collections import defaultdict

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# django.setup()

In [3]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [4]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)

# Generates event
event_defaults = {
    "members": []
}
event, __ = Event.objects.get_or_create(
    name="Hockey Party",
)
event.save()

# Connects users to event
UserEvent.objects.update_or_create(
    user=andrew,
    event=event,
    defaults={
        "is_host": True,
        "is_required": True
    }
)
UserEvent.objects.update_or_create(
    user=jessie,
    event=event,
    defaults={
        "is_host": False,
        "is_required": True
    }
)

# Generates proposed times
proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]
for proposed_start, proposed_end in proposed_times:
    EventTime.objects.update_or_create(
        event=event,
        date_time_start=proposed_start,
        defaults={"date_time_end": proposed_end},
    )

# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,11,1,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,12,1,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,13,1,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [6]:
def find_available_unseen_suggested_date(user_event):
    """
    Finds an UserEventTime that works for all Event participants
    """
    
    # Filters for event_times that work for everybody
    __, viable_event_times = check_availabilities(user_event.event)
    
    # Filters for unseen user_event_times
    valid_user_event_times = list(UserEventTime.objects.filter(
        user=user_event.user,
        event_time__in=viable_event_times,
        has_seen=False,
        is_active=False
    ))
    
    return valid_user_event_times

# find_available_unseen_suggested_date(user_event)

In [7]:



def send_suggestion_text(user_event_time):
    """
    """
    # Extracts data
    event = user_event_time.event_time.event
    user = user_event_time.user
    event_time_start = user_event_time.event_time.date_time_start
    event_time_end = user_event_time.event_time.date_time_end
    
    # Sends text
    print(f"Would {event_time_start} to {event_time_end} work for you?")
    
    # Updates user event time status
    user_event_time.is_active = True
    user_event_time.has_seen = True
    user_event_time.save()
    
    # Updates users event status: WAITING_RESPONSE
    user_event = UserEvent.objects.get(
        user=user,
        event=event
    )
    user_event.state = UserEvent.WAITING_RESPONSE
    user_event.save()
    
    return user_event.state


def create_response(user_event, input_text=None):
    """
    """
    # Gets User's status with respect to the Event
    user_event_state = user_event.state
    
    # If the user has not seen invite to event
    if user_event_state == UserEvent.NO_COMMUNICATION:
        # Finds a time that the user has not seen yet
        unseen_user_event_times = find_available_unseen_suggested_date(user_event)
        
        # Sends suggested EventTime
        user_event_state = send_suggestion_text(unseen_user_event_times[0])

    # If we are waiting for a response
    elif user_event_state == UserEvent.WAITING_RESPONSE:
        # Process input text
        explicit_response = UserEventTime.CAN_COME if strtobool(input_text) else UserEventTime.CANNOT_COME
        
        # Updates old UserEventTime state
        user_event_time = UserEventTime.objects.get(
            user=user_event.user,
            event_time__event=user_event.event,
            is_active=True,
            has_seen=True
        )
        user_event_time.is_active = False
        user_event_time.explicit_response = explicit_response
        user_event_time.save()

        # Look for more unseen new UserEventTime
        unseen_user_event_times = find_available_unseen_suggested_date(user_event)
        has_unseen_user_event_times = len(unseen_user_event_times) > 0
        if has_unseen_user_event_times:
            user_event_state = send_suggestion_text(unseen_user_event_times[0])
        else:
            __, viable_event_times = check_availabilities(event)
            no_viable_event_times = len(viable_event_times) < 1
            if no_viable_event_times:
                print("There are no viable event times, can you please suggest a time that works for you?")
            else:
                print(viable_event_times)
                user_event_state = UserEvent.WAITING_FOR_OTHERS
                user_event.state = user_event_state
                user_event.save()
    
    return user_event_state

# Reset
uets = UserEventTime.objects.filter(
    user=jessie,
    event_time__event=event
)

for uet in uets:
    uet.is_active=False
    uet.has_seen=False
    uet.save()

user_event = UserEvent.objects.get(user=jessie, event=event)
user_event.state = UserEvent.NO_COMMUNICATION
user_event.save()


# State 1: NO_RESPONSE
user_event = UserEvent.objects.get(user=jessie, event=event)
new_user_event_state = create_response(user_event)
print(new_user_event_state)
print("---------------------------------")

# State 2: WAITING RESPONSE for date suggestion
user_event = UserEvent.objects.get(user=jessie, event=event)
new_user_event_state = create_response(user_event, "No")
print(new_user_event_state)




IndexError: list index out of range

In [8]:
pd.DataFrame(UserEvent.objects.values())

,id,state,user_id,event_id,is_required,is_host
0,4,None,1,1,True,True
1,5,no_communication,2,1,True,False


In [9]:
pd.DataFrame(UserEventTime.objects.values())

,id,event_time_id,user_id,is_active,has_seen,explicit_response
0,7,11,1,False,False,waiting_response
1,8,12,1,False,False,waiting_response
2,9,13,1,False,False,waiting_response
3,10,11,2,False,False,waiting_response
4,11,12,2,False,False,waiting_response
5,12,13,2,False,False,cannot_come


In [ ]:
check_availabilities(event)

In [ ]:
def process_suggested_date(user, event, suggested_event_time):
    """
    Compares suggested_event_time against all members in event
    """
    participants = get_all_event_participants(event)
    availability_dict, times_that_work = check_availabilities(event)
    
    time_works = suggested_event_time.id in times_that_work
    available_participants = availability_dict[suggested_event_time.id]
    unavailable_participants = list(set(participants) - set(available_participants))
        
    return time_works, unavailable_participants
    

user = andrew
suggested_event_time = EventTime.objects.get(
    event_id=event.id,
    date_time_start=FEB_5_2PM
)

process_suggested_date(user, event, suggested_event_time)

In [ ]:
event_time = EventTime.objects.get(id=11)

In [ ]:
user_event_times = UserEventTime.objects.filter(
    event_time=event_time,
    user__in=[andrew, jessie],
).select_related("user")

user_event_times

In [ ]:
pd.DataFrame(UserEventTime.objects.values())